In [24]:
from langchain.chat_models import ChatOpenAI
from typing import Text
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

cache_dir = LocalFileStore("./.cache/")

llm = ChatOpenAI(temperature=0.1)

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="Commentaires",
)


loader = UnstructuredFileLoader("./files/plan_weekly.xlsx")
docs = loader.load_and_split(text_splitter=splitter)


In [20]:
for doc in docs:
    doc.page_content = doc.page_content.replace('\n', '')


In [25]:
embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir)
vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer, just say you don't know. Don't make it up:\n{context}"),
    ("human", "{question}")
])

chain = {"context": retriever, "question": RunnablePassthrough(
), "extra": RunnablePassthrough()} | prompt | llm

chain.invoke("목요일에 무슨 일이 있었습니까")
